In [5]:
# minimal output from pythons https://stackoverflow.com/a/78281950/3136861
%xmode Minimal  

Exception reporting mode: Minimal


In [6]:
%%bash
# setup playground

sh -c "[[ -d ./tmp ]] && { rm -r ./tmp; mkdir ./tmp;} || { mkdir -p ./tmp; }"
# The above oneliner removes the previous ./tmp/ and its content, then creates a new one with the same name
# It is modified after:
# stackoverflow.com/questions/73557323/bash-one-liner-check-if-a-directory-exists-if-so-delete-it-then-create-it-an

touch ./tmp/a{0..1}.txt # create some sample file in ./tmp/ folder

echo '*' > ./tmp/.gitignore

tree -a ./tmp

./tmp
├── .gitignore
├── a0.txt
└── a1.txt

1 directory, 3 files


# Quoting

[Reference to bash manual](https://www.gnu.org/software/bash/manual/html_node/Quoting.html)

Some shell characters have special meanings to the shell. For example, the [metacharacter](https://www.gnu.org/software/bash/manual/html_node/Definitions.html#index-metacharacter) (space, tab, newline, etc) is used to separate words. "Quoting" in bash means removing the special meaning of certain characters or words so that these characters can represent themselves literally.

There are three quoting mechanisms:
1. **strong quoting** with single-quotes
2. per-character **escaping** using a backslash
3. **weak quoting** with double-quote

File under the [section](https://www.gnu.org/software/bash/manual/html_node/Quoting.html) with quoting in the bash manual are: 

4. ANSI-C Quoting
5. Locale-Specific Tranlation

These quotes differ from the strong, escape, and weak quotes we mentioned earlier. Strong, escape and weak quotes **remove** the special meaning of some characters or words, but the ANSI-C Quoting and the Locale-Specific Translation enforce special interpretations.   

## 1. Single quotes (strong quoting)

Enclosing characters in single quotes `'` preserves the literal value of each character within the quotes. 

In [7]:
%%bash
echo tmp/*
echo tmp/'*'

tmp/a0.txt tmp/a1.txt
tmp/*


Note that a single quote may not occur between single quotes: 
```
echo 'please don't do this'   #error
```

Even when preceded by a backslash:
```
echo 'please don\'t do this' #error
```
This is because the escape character `\` is treated literally inside a single quote and thus loses its ability to escape the single quote character:

In [8]:
%%bash
echo '\*'

\*


Ref: [gnu bash manual](https://www.gnu.org/software/bash/manual/html_node/Single-Quotes.html)

## 2. Per-character escaping with a backslash

A *non-quoted* backslash `\` is the Bash **escape character**. The escape character preserves the literal value of the next character that follows, for example: 


In [9]:
%%bash
echo \'
echo \"
echo \\

'
"
\


It should be emphasized that by *non-quoted* we mean none of the following:

```
'\'   # escape character is within a pair of strong quote
"\"   # escape character is within a pair of weak quote
\\    # escape character is "escaped" by another escape character
```

When the backslash is *quoted* as shown above, it loses its meaning and is no longer an escape character.

### line-continuation with backslash

There is one exception when the backslash does not preserve the literal value of the next character. That is when a backslash is followed by the  *newline* character:

In [10]:
%%bash
echo this line will continue \
on next line.

this line will continue on next line.


In the terminal, when bash is run interactively, the *newline* character is the action of pressing the "Enter" button (or "Return" key) on your keyboard, which creates a new line in the terminal.

In a shell script (a text file containing shell commands), the *newline* character is the Linux new line character. 

Since the *newline* character is not usually displayed in the script editor, the line-continuation (i.e., the `\`+`newline` pair) could be separated by trailing space, resulting in a hard-to-find error [[see illustration of this error](./ipynb--line-continuation-error-due-to-trailing-space/index.ipynb)].

## 3. Double quote (weak quoate)

Enclosing characters in double quotes `"` preserves the literal value of all characters within the quotes, except for:

3a. `$`
3b. `` ` ``
3c. `\` 
3d. `!` (when history expansion is enabled and not in POSIX mode)

### 3a. `$`

Within the double quote, the `$` character preserves its special meaning. For example, parameter expansion, command substitution, or arithmetic expansion will continue to function even within the double quote:

```
"$foo", "${foo}"     # parameter expansion
"$(command)"         # command substituion
"$(( expression ))"  # Arithmetic Expansion

```

### 3b. `` ` ``

Backtick is the old-style syntax for command substitution:

```
"`command`"
```

Note that enclosing a single backtick in a double quote causes error:

In [15]:
%%bash
echo "`" 

bash: line 1: unexpected EOF while looking for matching ``'


CalledProcessError: Command 'b'echo "`" \n'' returned non-zero exit status 2.

### 3c. `\` 

As seen earlier, the backslash is the per-character escaping prefix; However, within the double quote backslash can only be use to excape `$`, `` ` ``, `"`, `\`, or `newline`:

In [45]:
%%bash
a=foo
echo "$a"   # here parameter expansion is working, the double quote does not turn off $'s functionality
echo "\$a"  # when prefix the $, it turn off the meaning of $, parameter expansion no longer happen
echo "\a"   # when not prefixing $, `, ", \, or newline, double quoted backslash is literally a backslash 
echo \a     # without quoting backslash preserves the literal value of the next character that follows

foo
$a
\a
a


Note that enclosing a single backslash in double quote cause error:

In [18]:
%%bash
echo "\"

bash: line 1: unexpected EOF while looking for matching `"'


CalledProcessError: Command 'b'echo "\\"\n'' returned non-zero exit status 2.

### 3d. `!`

Double quote will not suppress history expansion.

In [ ]:
As point out above the following 4, and 5 enforce special interpretations. 

## 4. ANSI-C Quoting

The $'...' syntax in Bash processes escape sequences by interpreting them according to ANSI C conventions. This means any valid escape sequence within the string is replaced with the corresponding character. Here's what the processing involves:

Examples of Escape Sequences and Their Processing:

| Escape Sequence | Meaning | Resulting Character | 
| --- | --- | --- | 
| `\n` | Newline | Line break | 
| `\t` | Tab | Horizontal tab | 
|`\\` | Backslash | `\` | 
| `\'` | Single quote | `'` | 
| `\"` | Double quote | `"` | 
| `\xHH` | Hexadecimal character | ASCII from hex code | 
| `\uHHHH` | Unicode character (16-bit) | Unicode char | 
| `\UHHHHHHHH` | Unicode character (32-bit) | Unicode char |


## 5. Locale-Specific Translation